In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn import linear_model
import numpy as np, statsmodels.stats.api as sms

In [2]:
Pokedex = pd.read_csv('pokedex_(Update_05.20).csv')

In [3]:
Pokedex = Pokedex.drop(['Unnamed: 0'], axis=1)
Pokedex.head()

,pokedex_number,name,german_name,japanese_name,generation,status,species,type_number,type_1,type_2,...,against_ground,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy
0,1,Bulbasaur,Bisasam,フシギダネ (Fushigidane),1,Normal,Seed Pokémon,2,Grass,Poison,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
1,2,Ivysaur,Bisaknosp,フシギソウ (Fushigisou),1,Normal,Seed Pokémon,2,Grass,Poison,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
2,3,Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,Poison,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
3,3,Mega Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,Poison,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
4,4,Charmander,Glumanda,ヒトカゲ (Hitokage),1,Normal,Lizard Pokémon,1,Fire,NaN,...,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5


In [4]:
Pokedex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 50 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pokedex_number    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   german_name       938 non-null    object 
 3   japanese_name     938 non-null    object 
 4   generation        1028 non-null   int64  
 5   status            1028 non-null   object 
 6   species           1028 non-null   object 
 7   type_number       1028 non-null   int64  
 8   type_1            1028 non-null   object 
 9   type_2            542 non-null    object 
 10  height_m          1028 non-null   float64
 11  weight_kg         1027 non-null   float64
 12  abilities_number  1028 non-null   int64  
 13  ability_1         1025 non-null   object 
 14  ability_2         513 non-null    object 
 15  ability_hidden    810 non-null    object 
 16  total_points      1028 non-null   float64


In [5]:
nulls_summary = pd.DataFrame(Pokedex.isnull().any(), columns=['Nulls'])   
nulls_summary['Num_of_nulls [qty]'] = pd.DataFrame(Pokedex.isnull().sum())   
nulls_summary['Num_of_nulls [%]'] = round((Pokedex.isnull().mean()*100),2)   
print(nulls_summary) 

                  Nulls  Num_of_nulls [qty]  Num_of_nulls [%]
pokedex_number    False                   0              0.00
name              False                   0              0.00
german_name        True                  90              8.75
japanese_name      True                  90              8.75
generation        False                   0              0.00
status            False                   0              0.00
species           False                   0              0.00
type_number       False                   0              0.00
type_1            False                   0              0.00
type_2             True                 486             47.28
height_m          False                   0              0.00
weight_kg          True                   1              0.10
abilities_number  False                   0              0.00
ability_1          True                   3              0.29
ability_2          True                 515             50.10
ability_

In [6]:
Pokedex[Pokedex['german_name'].isna()]

,pokedex_number,name,german_name,japanese_name,generation,status,species,type_number,type_1,type_2,...,against_ground,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy
938,808,Meltan,NaN,NaN,7,Mythical,Hex Nut Pokémon,1,Steel,NaN,...,2.0,0.5,0.5,0.50,0.50,1.0,0.5,1.0,0.5,0.5
939,809,Melmetal,NaN,NaN,7,Mythical,Hex Nut Pokémon,1,Steel,NaN,...,2.0,0.5,0.5,0.50,0.50,1.0,0.5,1.0,0.5,0.5
940,810,Grookey,NaN,NaN,8,Normal,Chimp Pokémon,1,Grass,NaN,...,0.5,2.0,1.0,2.00,1.00,1.0,1.0,1.0,1.0,1.0
941,811,Thwackey,NaN,NaN,8,Normal,Beat Pokémon,1,Grass,NaN,...,0.5,2.0,1.0,2.00,1.00,1.0,1.0,1.0,1.0,1.0
942,812,Rillaboom,NaN,NaN,8,Normal,Drummer Pokémon,1,Grass,NaN,...,0.5,2.0,1.0,2.00,1.00,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,888,Zacian Hero of Many Battles,NaN,NaN,8,Legendary,Warrior Pokémon,1,Fairy,NaN,...,1.0,1.0,1.0,0.50,1.00,1.0,0.0,0.5,2.0,1.0
1024,889,Zamazenta Crowned Shield,NaN,NaN,8,Legendary,Warrior Pokémon,2,Fighting,Steel,...,2.0,1.0,1.0,0.25,0.25,1.0,0.5,0.5,0.5,1.0
1025,889,Zamazenta Hero of Many Battles,NaN,NaN,8,Legendary,Warrior Pokémon,1,Fighting,NaN,...,1.0,2.0,2.0,0.50,0.50,1.0,1.0,0.5,1.0,2.0
1026,890,Eternatus,NaN,NaN,8,Legendary,Gigantic Pokémon,2,Poison,Dragon,...,2.0,1.0,2.0,0.50,1.00,1.0,2.0,1.0,1.0,1.0


In [7]:
Pokedex = Pokedex.drop(['german_name'], axis=1)
Pokedex = Pokedex.drop(['japanese_name'], axis=1)

In [8]:
Zero_count = pd.DataFrame(Pokedex.isin([0]).any(), columns=['Zero'])   
Zero_count['Num_of_zero [qty]'] = pd.DataFrame(Pokedex.isin([0]).sum())   
Zero_count['Num_of_zero [%]'] = round((Pokedex.isin([0]).mean()*100),2)   
print(Zero_count) 

                   Zero  Num_of_zero [qty]  Num_of_zero [%]
pokedex_number    False                  0             0.00
name              False                  0             0.00
generation        False                  0             0.00
status            False                  0             0.00
species           False                  0             0.00
type_number       False                  0             0.00
type_1            False                  0             0.00
type_2            False                  0             0.00
height_m          False                  0             0.00
weight_kg         False                  0             0.00
abilities_number   True                  1             0.10
ability_1         False                  0             0.00
ability_2         False                  0             0.00
ability_hidden    False                  0             0.00
total_points      False                  0             0.00
hp                False                 

,count,mean,std,min,25%,50%,75%,max
pokedex_number,1028.0,437.711089,259.366480,1.00,213.75,433.5,663.250,890.0
generation,1028.0,4.034047,2.234937,1.00,2.00,4.0,6.000,8.0
type_number,1028.0,1.527237,0.499501,1.00,1.00,2.0,2.000,2.0
height_m,1028.0,1.368093,3.380126,0.10,0.60,1.0,1.500,100.0
weight_kg,1027.0,69.753749,129.221230,0.10,8.80,28.5,69.100,999.9
abilities_number,1028.0,2.284047,0.794981,0.00,2.00,2.0,3.000,3.0
total_points,1028.0,437.571984,121.664910,175.00,330.00,455.0,510.000,1125.0
hp,1028.0,69.577821,26.385849,1.00,50.00,66.5,80.000,255.0
attack,1028.0,80.119650,32.372321,5.00,55.00,76.0,100.000,190.0
defense,1028.0,74.475681,31.303309,5.00,50.00,70.0,90.000,250.0


In [10]:
Pokedex['type_2'] = Pokedex['type_2'].fillna('Ninguno')
Pokedex['ability_2'] = Pokedex['ability_2'].fillna('Ninguno')
Pokedex['ability_hidden'] = Pokedex['ability_hidden'].fillna('Ninguno')
Pokedex['egg_type_2'] = Pokedex['egg_type_2'].fillna('Ninguno')


In [11]:
Pokedex['catch_rate'].fillna(Pokedex['catch_rate'].mean(), inplace=True)
Pokedex['base_experience'].fillna(Pokedex['base_experience'].mean(), inplace=True)
Pokedex['base_friendship'].fillna(Pokedex['base_friendship'].mean(), inplace=True)
Pokedex['percentage_male'].fillna(Pokedex['percentage_male'].mean(), inplace=True)
Pokedex['weight_kg'].fillna(Pokedex['weight_kg'].mean(), inplace=True)

In [12]:
Pokedex2 = Pokedex.dropna() 

In [25]:
Q1 = Pokedex2.quantile(0.25)
Q3 = Pokedex2.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

pokedex_number      449.250000
generation            4.000000
type_number           1.000000
height_m              0.900000
weight_kg            59.775000
abilities_number      1.000000
total_points        180.000000
hp                   30.000000
attack               45.000000
defense              40.000000
sp_attack            45.000000
sp_defense           40.000000
speed                45.000000
catch_rate           75.000000
base_friendship       0.000000
base_experience     113.000000
egg_type_number       1.000000
percentage_male       5.003157
egg_cycles            5.000000
against_normal        0.000000
against_fire          1.500000
against_water         0.500000
against_electric      0.500000
against_grass         0.500000
against_ice           1.500000
against_fight         1.500000
against_poison        0.500000
against_ground        1.000000
against_flying        0.000000
against_psychic       0.000000
against_bug           0.500000
against_rock          1.000000
against_

In [26]:
Pokedex2.describe().T

,count,mean,std,min,25%,50%,75%,max
pokedex_number,1024.0,437.855469,258.965844,1.00,214.000,433.500000,663.250000,890.0
generation,1024.0,4.035156,2.231634,1.00,2.000,4.000000,6.000000,8.0
type_number,1024.0,1.527344,0.499496,1.00,1.000,2.000000,2.000000,2.0
height_m,1024.0,1.273437,1.396168,0.10,0.600,1.000000,1.500000,20.0
weight_kg,1024.0,69.828711,129.370540,0.10,8.875,28.650000,68.650000,999.9
abilities_number,1024.0,2.289062,0.791226,1.00,2.000,2.000000,3.000000,3.0
total_points,1024.0,436.810547,119.947476,175.00,330.000,455.000000,510.000000,780.0
hp,1024.0,69.390625,25.757607,1.00,50.000,66.500000,80.000000,255.0
attack,1024.0,80.012695,32.320288,5.00,55.000,76.000000,100.000000,190.0
defense,1024.0,74.351562,30.864561,5.00,50.000,70.000000,90.000000,230.0


In [13]:
Pokedex2.to_csv("Pokedex.csv")

In [14]:
Pokedex_filter = Pokedex2[[ 'name', 'generation', 'status', 'species',
       'type_number',  'height_m', 'weight_kg',
       'abilities_number',   'total_points', 'hp', 'attack', 'defense', 'sp_attack', 'sp_defense',
       'speed', 'catch_rate']]

In [15]:
# Shapiro-Wilk Test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
# seed the random number generator
seed(1)
# generate univariate observations
data = Pokedex[['generation',
       'type_number',  'height_m', 'weight_kg',
       'abilities_number',       'total_points', 'hp', 'attack', 'defense', 'sp_attack', 'sp_defense',
       'speed', 'catch_rate']]
# normality test
for i in list(data.columns):
    var = data[i]
    print(i)
    stat, p = shapiro(var)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')



generation
Statistics=0.925, p=0.000
Sample does not look Gaussian (reject H0)
type_number
Statistics=0.636, p=0.000
Sample does not look Gaussian (reject H0)
height_m
Statistics=0.161, p=0.000
Sample does not look Gaussian (reject H0)
weight_kg
Statistics=0.511, p=0.000
Sample does not look Gaussian (reject H0)
abilities_number
Statistics=0.759, p=0.000
Sample does not look Gaussian (reject H0)
total_points
Statistics=0.977, p=0.000
Sample does not look Gaussian (reject H0)
hp
Statistics=0.894, p=0.000
Sample does not look Gaussian (reject H0)
attack
Statistics=0.982, p=0.000
Sample does not look Gaussian (reject H0)
defense
Statistics=0.936, p=0.000
Sample does not look Gaussian (reject H0)
sp_attack
Statistics=0.960, p=0.000
Sample does not look Gaussian (reject H0)
sp_defense
Statistics=0.955, p=0.000
Sample does not look Gaussian (reject H0)
speed
Statistics=0.983, p=0.000
Sample does not look Gaussian (reject H0)
catch_rate
Statistics=0.856, p=0.000
Sample does not look Gaussian 

In [29]:
 df = pd.DataFrame(Pokedex2[['generation', 'status', 'species', 'type_number',  'height_m', 'weight_kg',
       'abilities_number', 
'hp', 'attack', 'defense', 'sp_attack', 'sp_defense',
       'speed']])

target = pd.DataFrame(Pokedex_filter.catch_rate)

In [30]:
df = pd.get_dummies(df, columns=['status', 'species'])

In [31]:
lm = linear_model.LinearRegression()
model = lm.fit(df,target)

In [32]:
regressor = LinearRegression(fit_intercept=False)
regressor.fit(df, target)

regressor.coef_ 


array([[-5.08040810e+00, -8.73350999e+00,  1.14618307e+00,
         1.09890609e-01, -9.19173219e+00, -5.52164266e-01,
        -3.93326051e-01, -4.01817337e-01, -2.48414008e-01,
        -6.23010344e-01, -4.77422536e-01,  2.76991142e+02,
         3.14304769e+02,  3.37403505e+02,  3.11135690e+02,
         8.47879414e-01,  6.65096578e+01,  5.87628811e+01,
         1.16478702e+01,  5.83225910e+01,  9.29458367e+01,
         1.61700824e+01,  1.10444573e+02, -4.44597035e+01,
         5.48880778e+01,  4.94625886e+01,  2.82315694e+01,
        -1.56043904e+01,  1.05628096e+02, -5.41204090e+01,
        -3.75384947e+00,  9.00695981e+00,  3.64504837e+00,
        -1.06272077e+01, -2.14327257e+01,  6.53881842e+01,
         2.56807242e-01, -2.81905698e+01,  2.67519639e+01,
        -8.65079158e+00, -2.03541175e+01, -5.48000286e+01,
        -3.58340310e+01,  9.72982172e+00,  1.71400663e+01,
         3.78575999e+00, -1.65451685e+01, -2.40338348e+01,
        -4.12849166e+01, -1.36474024e+01,  4.04714927e+0

In [33]:
predictions = lm.predict(df)
print(predictions)

[[117.36132812]
 [ 79.11914062]
 [ 35.90625   ]
 ...
 [120.65625   ]
 [ 92.18945312]
 [ 93.15039062]]


In [34]:
r2_score(target, predictions)

0.8686975060361204

In [35]:

df = pd.DataFrame(Pokedex2[['generation', 'type_number',  'height_m', 'weight_kg',
       'abilities_number', 
'hp', 'attack', 'defense', 'sp_attack', 'sp_defense',
       'speed']])


target = pd.DataFrame(Pokedex_filter.catch_rate)

lm = linear_model.LinearRegression()
model = lm.fit(df,target)
predictions = lm.predict(df)

In [36]:
regressor = LinearRegression(fit_intercept=False)
regressor.fit(df, target)

regressor.coef_ 


array([[ 4.53292809e+00,  2.24023766e+01, -1.23651182e+00,
        -1.40567087e-02,  4.32438795e+01, -2.21626346e-01,
        -1.36663981e-01, -8.82081969e-02, -1.53624755e-01,
        -2.04668302e-01, -3.51699408e-02]])

In [37]:
r2_score(target, predictions)

0.4909573829767956

In [38]:
cm = sms.CompareMeans(sms.DescrStatsW(Pokedex_filter[Pokedex_filter['status']=='Legendary'].total_points), sms.DescrStatsW(Pokedex_filter[Pokedex_filter['status']=='Normal'].total_points))
print (cm.tconfint_diff(usevar='unequal'))

(217.5528519004007, 288.06118318731865)
